# Data Formatting

In [1]:
import tensorflow as tf
import yaml
import os
from object_detection.utils import dataset_util
import glob
import numpy as np
from PIL import Image

/home/camilog/anaconda3/envs/tf13/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
DATA_PATH = '/home/camilog/Downloads/dataset-sdcnd-capstone/data/'

## Categories in dataset

In [3]:
LABEL_DICT =  {
    "Green" : 1,
    "Red" : 2,
    "Yellow" : 3,
    "no_light" : 4,
    }

## Output File as TFRecord

In [4]:
OUTPUT_PATH = DATA_PATH + 'train_data.record'

In [5]:
YAML_FILES = []
YAML_FILES.append(DATA_PATH + 'real_training_data/real_data_annotations.yaml')
YAML_FILES.append(DATA_PATH + 'sim_training_data/sim_data_annotations.yaml')

examples = []
for i, file_name in enumerate(YAML_FILES):
    temp = yaml.load(open(file_name, 'rb').read())
    for i in range(len(temp)):
        temp[i]['path'] = os.path.abspath(os.path.join(os.path.dirname(file_name), temp[i]['filename']))
    
    examples += temp
    print('Samples in yaml %d: %d' % (i+1, len(temp)))

len_examples = len(examples)
print("Loading ", len_examples, "examples")

Samples in yaml 159: 159
Samples in yaml 277: 277
Loading  436 examples


### Adding real 'no_light' examples manually

In [6]:
PATH_TO_NOLIGHT_SAMPLES = DATA_PATH + 'real_training_data/nolight/'

nolight_samples = []
for file in glob.glob(PATH_TO_NOLIGHT_SAMPLES + "*.jpg"):
    temp = {'annotations': [],
             'class': 'image',
             'path': PATH_TO_NOLIGHT_SAMPLES + file.split('/')[-1]}
    nolight_samples.append(temp)
    
examples += nolight_samples

In [7]:
labels = ['no_light']
counter = {'no_light': 0}
for sample in examples:
    annotations = sample['annotations']
    if len(annotations) == 0:
        counter['no_light'] += 1
    else:
        for annotation in annotations:
            if annotation['class'] not in labels:
                labels.append(annotation['class'])
                counter[annotation['class']] = 1
            else:
                counter[annotation['class']] += 1
            
print('Number of annotations per category:')
print(counter)

Number of annotations per category:
{'no_light': 105, 'Red': 504, 'Green': 178, 'Yellow': 147}


## Formatting Data: 

In [8]:
def create_tf_example(example):
    
    jpgfile = Image.open(example['path'])
    width, height = jpgfile.size

    filename = example['path'] # Filename of the image. 
    filename = filename.encode()

    with tf.gfile.GFile(example['path'], 'rb') as fid:
        encoded_image = fid.read()

    image_format = 'jpg'.encode() 

    xmins = [] # List of normalized left x coordinates in bounding box (1 per box)
    xmaxs = [] # List of normalized right x coordinates in bounding box (1 per box)
    ymins = [] # List of normalized top y coordinates in bounding box (1 per box)
    ymaxs = [] # List of normalized bottom y coordinates in bounding box (1 per box)
    classes_text = [] # List of string class name of bounding box (1 per box)
    classes = [] # List of integer class id of bounding box (1 per box)

    for box in example['annotations']:
        xmins.append(float(box['xmin'] / width))
        xmaxs.append(float((box['xmin'] + box['x_width']) / width))
        ymins.append(float(box['ymin'] / height))
        ymaxs.append(float((box['ymin'] + box['y_height']) / height))
        classes_text.append(box['class'].encode())
        classes.append(int(LABEL_DICT[box['class']]))


    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_image),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))

    return tf_example


In [9]:
writer = tf.python_io.TFRecordWriter(OUTPUT_PATH)

counter = 0
for example in examples:
    tf_example = create_tf_example(example)
    writer.write(tf_example.SerializeToString())

    if counter % 100 == 0:
        print("Percent done", (counter/len_examples)*100)
    counter += 1

writer.close()
print("Done!")

Percent done 0.0
Percent done 22.93577981651376
Percent done 45.87155963302752
Percent done 68.80733944954129
Percent done 91.74311926605505
Done!
